In [ ]:
import os
import pandas as pd
import warnings
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.worksheet.table import Table, TableStyleInfo
from openpyxl.styles import PatternFill, Alignment, Font
warnings.filterwarnings('ignore')

In [4]:
%%sql
SELECT
    ano,
    mes,
    data,
    numeroNotaFiscal,
    destino,
    tipo,
    procedencia,
    estado,
    produto,
    unidade,
    unidade_peso,
    quantidade,
    volume
FROM
    (
        SELECT
            EXTRACT(YEAR FROM data) AS ano,
            EXTRACT(MONTH FROM data) AS mes,
            data,
            numeroNotaFiscal,
            destino,
            procedencia,
            produto,
            unidade,
            unidade_peso,
            quantidade,
            volume
        FROM divtec_notasfiscais

        UNION ALL

        SELECT
            EXTRACT(YEAR FROM data) AS ano,
            EXTRACT(MONTH FROM data) AS mes,
            data,
            numeroNotaFiscal,
            destino,
            procedencia,
            produto,
            unidade,
            unidade_peso,
            quantidade,
            volume
        FROM notas_temporarias
    ) AS uniao
LEFT JOIN
    cad_cidades ON cad_cidades.cidade = uniao.procedencia
LEFT JOIN
    cad_usuarios ON uniao.destino = cad_usuarios.matricula_nome
ORDER BY ano, mes, data, destino, numeroNotaFiscal, produto;


,ano,mes,data,numeroNotaFiscal,destino,tipo,procedencia,estado,produto,unidade,unidade_peso,quantidade,volume
0,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,04702-BROCOLIS,DZ,6.000,10.0,60.00
1,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07002-PEPINO SALADA,CX,20.000,18.0,360.00
2,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07003-PEPINO JAPONES,CX,20.000,5.0,100.00
3,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07201-PIMENTAO,CX,10.000,8.0,80.00
4,2024,1,2024-01-02,21096,0100003-TAKAYOSHI SATO,PRODUTOR,0143057-GRAVATAI (RS),RIO GRANDE DO SUL,04801-CHEIRO VERDE / TEMPERO,DZ,0.500,20.0,10.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321497,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01406-LARANJA SUCO,CX,20.000,80.0,1600.00
321498,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01603-LIMAO TAITI,CX,20.000,10.0,200.00
321499,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01901-MANGA,CX,20.000,20.0,400.00
321500,2025,4,2025-04-25,589645,0113827-VALDOMIRO STRATSBURG TIETBOHL,PRODUTOR,0143420-TRES FORQUILHAS (RS),RIO GRANDE DO SUL,05601-REPOLHO VERDE,CX,15.426,20.0,308.52


In [5]:
wb = Workbook()

sheet = wb.active
sheet.title = "Produto-Destino-Procedência"

wb.create_sheet("Destino-Procedência-Produto")
wb.create_sheet("Procedência-Produto-Destino")
wb.create_sheet("Tipo-Procedência-Produto")
wb.create_sheet("Estado-Produto")
wb.create_sheet("Produtores-Fora-RS")

<Worksheet "Produtores-Fora-RS">

### Verificação da Relação => Produto / Destino / Procedência

In [6]:
df2 = df.copy(deep=True)
df2 = df2.groupby(['produto','destino','procedencia'])['volume'].sum().reset_index()

aba_destino = wb["Produto-Destino-Procedência"]

for r in dataframe_to_rows(df2, index=False, header=True):
	aba_destino.append(r)

### Verificação da Relação => Destino / Procedência / Produto

In [7]:
df3 = df.copy(deep=True)
df3 = df3.groupby(['destino','procedencia','produto'])['volume'].sum().reset_index()

aba_destino = wb["Destino-Procedência-Produto"]
for r in dataframe_to_rows(df3, index=False, header=True):
	aba_destino.append(r)

### Verificação da Relação => Procedência / Produto / Destino

In [8]:
df4 = df.copy(deep=True)
df4 = df4.groupby(['procedencia','produto','destino'])['volume'].sum().reset_index()

aba_destino = wb["Procedência-Produto-Destino"]
for r in dataframe_to_rows(df4, index=False, header=True):
	aba_destino.append(r)

### Verificação da Relação => Estado / Produto

In [9]:
df5 = df.copy(deep=True)
df5 = df5.groupby(['estado','produto'])['volume'].sum().reset_index()

aba_destino = wb["Estado-Produto"]
for r in dataframe_to_rows(df5, index=False, header=True):
	aba_destino.append(r)

### Verificação da Relação => Tipo / Procedência / Produto

In [10]:
df6 = df.copy(deep=True)
df6 = df6.groupby(['tipo','procedencia','produto'])['volume'].sum().reset_index()

aba_destino = wb["Tipo-Procedência-Produto"]
for r in dataframe_to_rows(df6, index=False, header=True):
	aba_destino.append(r)

### Verificação da Relação => Produtores Fora do RS

In [11]:
df7 = df.copy(deep=True)
df7 = df7.loc[(df7['estado'] == 'RIO GRANDE DO SUL') & (df7['tipo'] == 'PRODUTOR')]
df7 = df7.groupby(['destino','estado','procedencia'])['volume'].sum().reset_index()

aba_destino = wb["Produtores-RS"]
for r in dataframe_to_rows(df7, index=False, header=True):
	aba_destino.append(r)

### OUTPUT => EXCEL

#### Ajustes e Personalização

In [12]:
def ajustar_largura(sheet):
	for coluna in sheet.columns:
		max_length = 0
		column = coluna[0].column_letter  # Obter a letra da coluna
		for cell in coluna:
			try:
				if len(str(cell.value)) > max_length:
					max_length = len(cell.value)
			except:
				pass
		adjusted_width = (max_length + 2)  # Um pequeno espaçamento
		sheet.column_dimensions[column].width = adjusted_width

# Função para aplicar estilo de tabela
def aplicar_tabela(sheet):
	# Encontrar o intervalo de dados
	last_row = sheet.max_row
	last_column = sheet.max_column
	data_range = f"A1:{sheet.cell(row=last_row, column=last_column).coordinate}"

	# Gerar um nome único para a tabela, baseado no nome da aba
	table_name = f"TabelaEstilo_{sheet.title}"

	# Criar uma tabela no intervalo de dados
	table = Table(displayName=table_name, ref=data_range)

	# Estilo da tabela (pode ser ajustado conforme desejado)
	style = TableStyleInfo(
		name="TableStyleMedium2",  # Estilo de tabela predefinido
		showFirstColumn=False,
		showLastColumn=False,
		showRowStripes=True,
		showColumnStripes=True
	)

	# Aplicar o estilo à tabela
	table.tableStyleInfo = style

	# Adicionar a tabela à planilha
	sheet.add_table(table)

	# Estilo do cabeçalho (verde escuro com texto branco centralizado)
	header_fill = PatternFill(start_color="006400", end_color="006400", fill_type="solid")  # Verde escuro
	header_font = Font(color="FFFFFF")  # Letras brancas
	header_alignment = Alignment(horizontal="center", vertical="center")

	for cell in sheet[1]:  # A primeira linha é o cabeçalho
		cell.fill = header_fill
		cell.font = header_font
		cell.alignment = header_alignment

	# Estilo para linhas alternadas (verde claro e branco)
	light_green_fill = PatternFill(start_color="cce0cc", end_color="cce0cc", fill_type="solid")  # Verde claro
	white_fill = PatternFill(start_color="FFFFFF", end_color="FFFFFF", fill_type="solid")  # Branco

	for row_idx, row in enumerate(sheet.iter_rows(min_row=2, max_row=last_row, min_col=1, max_col=last_column), start=2):
		# Aplicar o estilo alternado nas linhas
		if row_idx % 2 == 0:
			fill = light_green_fill  # Linhas pares (começando pela segunda linha)
		else:
			fill = white_fill  # Linhas ímpares

		for cell in row:
			cell.fill = fill  # Aplica a cor de fundo à célula

# Laço para aplicar ajustes e tabelas a todas as abas
for sheet in wb.worksheets:
	ajustar_largura(sheet)  # Ajustar largura das colunas
	aplicar_tabela(sheet)  # Aplicar tabela


In [13]:
try:
    nome_arquivo = "Verificações __Fechamento 2024.xlsx"

    if os.path.exists(nome_arquivo):
        os.remove(nome_arquivo)

    wb.save(nome_arquivo)
    print(f"Planilha '{nome_arquivo}' criada com sucesso!")
except Exception as e:
    print(f"Ocorreu um erro ao criar a planilha: {e}")

Planilha 'Verificações Fechamento 2024.xlsx' criada com sucesso!
